In [ ]:
!pip install ujson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
import random
import os
import gzip
import bz2
import csv
import ujson as json
import glob
import math

In [ ]:
def write_file(out_file,mkdir=True,binary=False):
  if mkdir:
    dir = os.path.split(out_file)[0]
    if dir:
      os.makedirs(dir,exist_ok=True)

  if binary:
    if out_file.endswith('.gz'):
      return gzip.open(out_file,'wb')
    elif out_file.endswith('.bz2'):
      return bz2.open(out_file,'wb')
    else:
      return open(out_file,'wb')

  else:
    if out_file.endswith('.gz'):
      return gzip.open(out_file,'wt',encoding='utf-8')
    elif out_file.endswith('.bz2'):
      return bz2.open(out_file,'wt',encoding='utf-8')
    else:
      return open(out_file,'w',encoding='utf-8')




In [ ]:
def read_file(in_file,binary=False,errors=None):

  if binary:
    if in_file.endswith('.gz'):
      return gzip.open(in_file,'rb')
    elif in_file.endswith('.bz2'):
      return bz2.open(in_file,'rb')
    else:
      return open(in_file,'rb')

  else:
    if in_file.endswith('.gz'):
      return gzip.open(in_file,'rt',encoding='utf-8',errors=errors)
    elif in_file.endswith('.bz2'):
      return bz2.open(in_file,'rt',encoding='utf-8',errors=errors)
    else:
      return open(in_file,'r',encoding='utf-8',errors=errors)


In [ ]:
def shuffle_blocks(it,*,block_size=20000,rand=random):
  assert block_size>=4
  block = []
  for i in it:
    block.append(i)
    if len(block)>=block_size:
      rand.shuffle(block)
      for _ in range(block_size//2):
        yield block.pop(-1)

  rand.shuffle(block)
  for b in block:
    yield b

In [ ]:
def expand_file(input,file_pattern='*',files=None):
  if type(input) is str:
    if ':' in input:
      input = input.split(':')
    else:
      input = [input]

  all_inputs = []
  if files is None:
    files = []

  for i in input:
    print(i)
    if i in files:
      continue
    if os.path.isdir(i):
      sub_file = glob.glob(i+"/**/"+file_pattern,recursive=True)
      sub_file = [f for f in sub_file if not os.path.isdir(f)]
      sub_file = [f for f in sub_file if f not in input and f not in files]
      all_inputs.extend(sub_file)
    else:
      all_inputs.append(i)

  all_inputs.sort()
  return all_inputs



In [ ]:
def j_lines(input,files=None,limit=0,report_every=100000,*,errors=None,shuffled=None):
  lst = [f for f in expand_file(input,"*.jsonl*",files) if not f.endswith('.lock')]

  return read_lines(lst,limit=limit,report_every=report_every,errors=errors,shuffled=shuffled)


In [ ]:
def read_lines(input,limit=0,report_every=100000,*,errors=None,shuffled=None):
  count =0
  input = expand_file(input)
  if shuffled:
    if type(shuffled) != random.Random:
      shuffled = random.Random()

    open_blocks = int(math.ceil(len(input)/32.0))
    for open_i in range(open_blocks):
      open_files = [read_file(i,errors=errors) for i in input[open_i::open_blocks]]
      while len(open_files)>0:
        fx = shuffled.randint(0,len(open_files)-1)
        next_l = open_files[fx].readline()
        if next_l:
          yield next_l
          count +=1

        else:
          open_files[fx].close()
          del open_files[fx]

  else:
    for i in input:
      with read_file(i,errors=errors) as fp:
        for l in fp:
          yield l
          count +=1
          if 0<limit<= count:
            return



## wikisql

In [ ]:
!pip install records

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openpyxl: filename=openpyxl-2.4.11-py2.py3-none-any.whl size=222821 sha256=7713fc63bef088e22cfc7ea4b295f7da389be38c7d718e4b6057678756317994
  Stored in directory: /root/.cache/pip/wheels/f6/0a/c7/3744d0f62b960ef9ff407e2c38975e058629ace1255e8bd95b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0df5c1fb6b73f9336d4de5bc31f3646ce9afd040e593adc1c5ca6dfeafa29ee2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built openpyxl docopt
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.2
    Uninstalling openpyxl-3.1.2:
      Successfully uninstalled openpyxl-3.1.2


In [ ]:
!pip install sqlalchemy==1.3.23

In [ ]:
!git clone https://github.com/salesforce/WikiSQL.git
import tarfile

file_path = 'WikiSQL/data.tar.bz2'

with tarfile.open(file_path, 'r:bz2') as tar:
    tar.extractall()

Cloning into 'WikiSQL'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 386 (delta 185), reused 154 (delta 154), pack-reused 194
Receiving objects: 100% (386/386), 50.72 MiB | 42.12 MiB/s, done.
Resolving deltas: 100% (212/212), done.


In [ ]:
import json
from tqdm import tqdm
from WikiSQL.dbengine import DBEngine
from WikiSQL.query import Query
from WikiSQL.common import count_lines
import os
import re
# from util.line_corpus import jsonl_lines, write_open
# from util.args_help import fill_from_args
# from datasets.wikisql.tables2agg_classify import write_agg_classify
# import logging

In [ ]:
def cell_value(cell,ans):
  if cell in ans or cell.lower() in ans:
    return True
  try:
    cell_f = float(cell)
    for a in ans:
      try:
        if cell_f == float(a):
          return True
      except:
        pass
  except:
    pass

  return False


In [ ]:
def classify(data_dir,split,*,exclude_header=False,sep_token='*'):
  with write_file(os.path.join(data_dir,f'{split}_agg_classify.jsonl')) as fo:
    for l in j_lines(os.path.join(data_dir,f'{split}_agg.jsonl')):
      data = json.loads(l)
      if not exclude_header:
        inst = {'id': data['id'], 'text_a': data['question'],'text_b':f'{sep_token}'.join(data['header']),'label':data['index']}
      else:
        inst = {'id': data['id'], 'text': data['question'],'label':data['index']}
      fo.write(json.dumps(inst)+'\n')

In [ ]:
def convert_queries(queries,tables,output_file,data_file,*,skip_aggregation=True, show_aggregation=False):
  tid2rows = dict()
  for l in tables:
    data = json.loads(l)
    tid = data['id']
    header = data['header']
    rows_or = data['rows']
    rows = []

    for r in rows_or:
      rows.append([str(cv) for cv in r])

    tid2rows[tid] = [[str(h) for h in header]] + rows

  with open(data_file, 'w') as f:
    json.dump(tid2rows, f)

  with write_file(output_file) as fo:
    for qid,l in enumerate(queries):
      data = json.loads(l)
      index = data['sql']['agg']

      if skip_aggregation and index!=0:
        continue

      table_id = data['table_id']
      rows = tid2rows[table_id]
      qtext = data['question']
      target_column = data['sql']['sel']
      condition_columns = [colndx for colndx,comp,val in data['sql']['conds']]
      answers = data['answer']
      rowids = data['rowids'] if 'rowids' in data else None

      data = dict()
      data['id'] = f'{qid}'
      data['question'] = qtext
      data['header'] = rows[0]
      data['rows'] = rows[1:]
      data['target_column'] = target_column
      data['condition_columns'] = condition_columns
      data['table_id'] = table_id
      data['index'] = index

      if rowids is not None:
        data['target_rows'] = rowids

      if index ==0:
        answers = [str(ans) for ans in answers]
        clean_ans = []

        for r in rows[1:]:
          if cell_value(r[target_column],answers):
            clean_ans.append(r[target_column])

        data['answers'] = list(set(clean_ans))

      else:
        data['answers'] = answers

      # if show_aggregation and rowids and len(rowids) >1 and index !=0:

      fo.write(json.dumps(data)+'\n')





In [ ]:
from tqdm import tqdm

In [ ]:
for split in ['train','dev','test']:
  orig = os.path.join("data/",f'{split}.jsonl')
  db_file = os.path.join("data/",f'{split}.db')
  ans_file = os.path.join("data/",f'{split}_ans.jsonl.gz')
  tbl_file = os.path.join("data/",f'{split}.tables.jsonl')
  engine = DBEngine(db_file)

  with open(orig) as fs, write_file(ans_file) as fo:

    for ls in tqdm(fs):
      try:
        eg = json.loads(ls)
      except json.JSONDecodeError as e:
        continue
      sql = eg['sql']
      qg = Query.from_dict(sql,ordered=False)
      gold = engine.execute_query(eg['table_id'],qg,lower=True)
      assert isinstance(gold,list)

      eg['answer'] = gold
      eg['rowids'] = engine.execute_query_rowid(eg['table_id'],qg,lower=True)
      fo.write(json.dumps(eg)+'\n')

  convert_queries(j_lines(ans_file),j_lines(tbl_file),os.path.join("./", f"{split}_agg.jsonl"),"wiki_sql_data_agg.jsonl",skip_aggregation=False)
  convert_queries(j_lines(ans_file),j_lines(tbl_file),os.path.join("./", f"{split}_lookup.jsonl"),"wiki_sql_data_lookup.jsonl",skip_aggregation=True)
  classify("./",split)



56355it [07:30, 124.99it/s]


data/train_ans.jsonl.gz
data/train.tables.jsonl
data/train.tables.jsonl
data/train_ans.jsonl.gz
data/train_ans.jsonl.gz
data/train.tables.jsonl
data/train.tables.jsonl
data/train_ans.jsonl.gz
./train_agg.jsonl
./train_agg.jsonl


0it [00:00, ?it/s]ERROR:sqlalchemy.pool.impl.NullPool:Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 558, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 139873413885952 and this is thread id 139872938464832.
ERROR:sqlalchemy.pool.impl.NullPool:Exception closing connection <sqlite3.Connection object at 0x7f3692f8b440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/

data/dev_ans.jsonl.gz
data/dev.tables.jsonl
data/dev.tables.jsonl
data/dev_ans.jsonl.gz
data/dev_ans.jsonl.gz
data/dev.tables.jsonl
data/dev.tables.jsonl
data/dev_ans.jsonl.gz
./dev_agg.jsonl
./dev_agg.jsonl


37it [00:00, 365.14it/s]ERROR:sqlalchemy.pool.impl.NullPool:Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 558, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 139873413885952 and this is thread id 139872938464832.
ERROR:sqlalchemy.pool.impl.NullPool:Exception closing connection <sqlite3.Connection object at 0x7f3692f8b640>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/loca

data/test_ans.jsonl.gz
data/test.tables.jsonl
data/test.tables.jsonl
data/test_ans.jsonl.gz
data/test_ans.jsonl.gz
data/test.tables.jsonl
data/test.tables.jsonl
data/test_ans.jsonl.gz
./test_agg.jsonl
./test_agg.jsonl
